# **REDES NEURONALES**

Exportar librerias

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder

Cargar datos

In [3]:
data = pd.read_excel('/content/concursos1.xlsx')

Codificar las variables para poder ser uilizadas por el modelo

In [4]:
# Seleccionando las columnas para la codificación one-hot
categorical_features = ['tipodeentidadpadre', 'tipoentidad', 'modalidad', 'categorías']

# Inicializando el codificador
categorical_encoder = OneHotEncoder(sparse_output=False)

# Aplicar el OneHotEncoder a las características categóricas
X_categorical_encoded = categorical_encoder.fit_transform(data[categorical_features])
feature_names = categorical_encoder.get_feature_names_out(categorical_features)
X_categorical_encoded = pd.DataFrame(X_categorical_encoded, columns=feature_names)


In [5]:
# Normalizar las características codificadas
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X_categorical_encoded)
X_normalized = pd.DataFrame(X_normalized, columns=X_categorical_encoded.columns)

In [6]:
# Codificar la variable objetivo
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['entidadcompradora'])

Definición de set de entreno y prueba

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

Diseño de la red neuronal

In [8]:
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(np.unique(y_train).size, activation='softmax')  # Asegúrate de que el tamaño es correcto
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Compilación de la red

In [9]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Entreno y testeo

In [10]:
model.fit(X_train, y_train, epochs=10, batch_size=50, validation_data=(X_test, y_test))

Epoch 1/10
11430/11430 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - accuracy: 0.9982 - loss: 0.0405 - val_accuracy: 0.9992 - val_loss: 0.0046
Epoch 2/10
11430/11430 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.9993 - loss: 0.0042 - val_accuracy: 0.9992 - val_loss: 0.0047
Epoch 3/10
11430/11430 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - accuracy: 0.9994 - loss: 0.0040 - val_accuracy: 0.9992 - val_loss: 0.0046
Epoch 4/10
11430/11430 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.9993 - loss: 0.0047 - val_accuracy: 0.9992 - val_loss: 0.0049
Epoch 5/10
11430/11430 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - accuracy: 0.9993 - loss: 0.0043 - val_accuracy: 0.9992 - val_loss: 0.0049
Epoch 6/10
11430/11430 ━━━━━━━━━━━━━━━━━━━━ 25s 2ms/step - accuracy: 0.9993 - loss: 0.0056 - val_accuracy: 0.9992 - val_loss: 0.0045
Epoch 7/10
11430/11430 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - accuracy: 0.9994 - loss: 0.0045 - val_accuracy: 0.9992 - val_loss: 0.0047
Epoch 8/10
11430/11430 ━━━━━━━━━━━━━━━━━━━━ 43s 2ms/step - accuracy: 

Presición del modelo

In [11]:
loss, acc = model.evaluate(X_test, y_test)
print(acc*100)

4465/4465 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9992 - loss: 0.0049
99.91950392723083


Realizar predicciones

In [12]:
entidad_compradora = pd.DataFrame({
    'tipodeentidadpadre': ['Sector Público'],
    'tipoentidad': ['Administración Central'],
    'modalidad': ['Compra Directa con Oferta Electrónica (Art. 43 LCE Inciso b)'],
    'categorías': ['Salud e insumos hospitalarios']
})

Transformar el nuevo dato a catgorias

In [13]:
entidad_compradora_encoded = categorical_encoder.transform(entidad_compradora)
entidad_compradora_encoded = pd.DataFrame(entidad_compradora_encoded, columns=categorical_encoder.get_feature_names_out())



Predicciones

In [14]:
predictions = model.predict(entidad_compradora_encoded)
predicted_classes = np.argmax(predictions, axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_classes)

print("Categorías predichas:", predicted_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Categorías predichas: ['MINISTERIO DE SALUD PÚBLICA']


Segunda Predicción

In [15]:
entidad_compradora2 = pd.DataFrame({
    'tipodeentidadpadre': ['Sector Público'],
    'tipoentidad': ['Entidades Descentralizadas, Autónomas y de Seguridad Social'],
    'modalidad': ['Compra Directa con Oferta Electrónica (Art. 43 LCE Inciso b)'],
    'categorías': ['Salud e insumos hospitalarios']
})

In [16]:
entidad_compradora_encoded2 = categorical_encoder.transform(entidad_compradora2)
entidad_compradora_encoded2 = pd.DataFrame(entidad_compradora_encoded2, columns=categorical_encoder.get_feature_names_out())

In [17]:
predictions = model.predict(entidad_compradora_encoded2)
predicted_classes = np.argmax(predictions, axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_classes)

print("Categorías predichas:", predicted_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Categorías predichas: ['INSTITUTO GUATEMALTECO DE SEGURIDAD SOCIAL -IGSS-']
